In [108]:
#r "nuget:Google.Cloud.Storage.V1,4.13.0"
#r "nuget:System.Linq.Async,6.0.1"

Installed Packages Google.Cloud.Storage.V1, 4.13.0 System.Linq.Async, 6.0.1

In [109]:
using System;
using System.IO;
using System.Linq;
using Google.Cloud.Storage.V1;
using Google.Apis.Storage.v1.Data;


var client = new StorageClientBuilder()
{
    // storage client for .net will only work with http hosted emulator
    // also specifying -external-url option is required for the uploads to function properly
    // see attached docker-compose.yaml file for details
    UnauthenticatedAccess = true,
    BaseUri = "http://localhost:4443/storage/v1/",
}.Build();

string bucket = "ufs-test-bucket";


In [110]:
// create bucket
// client.CreateBucket("pid", bucket);

In [111]:
// list bucket
client.ListBuckets("pid")

[ Google.Apis.Storage.v1.Data.Bucket, Google.Apis.Storage.v1.Data.Bucket ] (values) index value 0 Google.Apis.Storage.v1.Data.Bucket Acl <null> Autoclass <null> Billing <null> Cors <null> CustomPlacementConfig <null> DefaultEventBasedHold False DefaultObjectAcl <null> Encryption <null> ETag RVRhZw== Generation <null> HardDeleteTimeRaw <null> HardDeleteTimeDateTimeOffset <null> HardDeleteTime <null> HierarchicalNamespace <null> IamConfiguration <null> Id abc IpFilter <null> Kind storage#bucket Labels <null> Lifecycle <null> Location US-CENTRAL1 LocationType region Logging <null> Metageneration 1 Name abc ObjectRetention <null> Owner <null> ProjectNumber 0 RetentionPolicy <null> Rpo <null> SatisfiesPZI <null> SatisfiesPZS <null> SelfLink <null> SoftDeletePolicy <null> SoftDeleteTimeRaw <null> SoftDeleteTimeDateTimeOffset <null> SoftDeleteTime <null> StorageClass STANDARD TimeCreatedRaw 2026-01-22T02:14:39.325172Z TimeCreatedDateTimeOffset 2026-01-22 02:14:39Z TimeCreated 2026-01-22 10:14:39Z UpdatedRaw 2026-01-22T02:14:39.325172Z UpdatedDateTimeOffset 2026-01-22 02:14:39Z Updated 2026-01-22 10:14:39Z Versioning Google.Apis.Storage.v1.Data.Bucket+VersioningData Enabled False Website <null> 1 Google.Apis.Storage.v1.Data.Bucket Acl <null> Autoclass <null> Billing <null> Cors <null> CustomPlacementConfig <null> DefaultEventBasedHold False DefaultObjectAcl <null> Encryption <null> ETag RVRhZw== Generation <null> HardDeleteTimeRaw <null> HardDeleteTimeDateTimeOffset <null> HardDeleteTime <null> HierarchicalNamespace <null> IamConfiguration <null> Id ufs-test-bucket IpFilter <null> Kind storage#bucket Labels <null> Lifecycle <null> Location US-CENTRAL1 LocationType region Logging <null> Metageneration 1 Name ufs-test-bucket ObjectRetention <null> Owner <null> ProjectNumber 0 RetentionPolicy <null> Rpo <null> SatisfiesPZI <null> SatisfiesPZS <null> SelfLink <null> SoftDeletePolicy <null> SoftDeleteTimeRaw <null> SoftDeleteTimeDateTimeOffset <null> SoftDeleteTime <null> StorageClass STANDARD TimeCreatedRaw 2026-01-22T02:19:45.464274Z TimeCreatedDateTimeOffset 2026-01-22 02:19:45Z TimeCreated 2026-01-22 10:19:45Z UpdatedRaw 2026-01-22T02:19:45.464274Z UpdatedDateTimeOffset 2026-01-22 02:19:45Z Updated 2026-01-22 10:19:45Z Versioning Google.Apis.Storage.v1.Data.Bucket+VersioningData Enabled False Website <null>

In [131]:
// add objects
string[] paths = new[]
{
    "file0.txt",
    "file00.txt",
    "file000.txt",
    "dir1/file1.txt",
    "dir1/dir2/file2.txt",
    "dir1/dir3/file3.txt",
    "dir1/dir3/dir4/file4.txt",
    "dir5/file5.txt",
    "dir5/dir6/file6.txt",
    "dir7/file7.txt",
};

foreach(string path in paths)
{
    client.UploadObject(bucket, path, "text/plain", 
        new MemoryStream(Encoding.UTF8.GetBytes("Hello Google Storage")));
}

In [140]:
// list objects
string token = null;

do
{
    ListObjectsOptions options = new()
    {
        Delimiter = "/",
        IncludeFoldersAsPrefixes = true,
        IncludeTrailingDelimiter = true,
        // PageSize = 2,
        PageToken = token,
        Fields = "items(name,contentType),nextPageToken",
    };


    var rawResponses = client.ListObjectsAsync(bucket, "", options).AsRawResponses();

    await using (var enumerator = rawResponses.GetAsyncEnumerator())
    {
        while (await enumerator.MoveNextAsync())
        {
            var response = enumerator.Current;
            Console.WriteLine($"--- Page ---");
            foreach (var obj in response.Items)
            {
                Console.WriteLine($"Object: {obj.Name}");
            }
            foreach (var prefix in response.Prefixes)
            {
                Console.WriteLine($"Prefix: {prefix}");
            }
        }
        token = enumerator.Current.NextPageToken;
    }
} while (token != null);


--- Page ---
Object: file0.txt
Object: file00.txt
Object: file000.txt
Prefix: dir1/
Prefix: dir5/
Prefix: dir7/


In [114]:
client.ListObjects(bucket, "").Select(x => x.Name)

[ dir1/dir2/file2.txt, dir1/dir3/dir4/file4.txt, dir1/dir3/file3.txt, dir1/file1.txt, dir5/dir6/file6.txt, dir5/file5.txt, file0.txt, file00.txt, file000.txt ]